# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading CSV into a dataframe
output_data = pd.read_csv("../output_data/cities.csv")
output_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring API key and gmaps
gmaps.configure(g_key)

In [4]:
# Using the Lat and Lng as locations
locations_df = output_data[['Lat','Lng']]
locations_df

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42
...,...,...
543,33.97,36.66
544,24.01,-104.61
545,49.65,0.71
546,51.07,132.56


In [5]:
# Getting Humidity as the weight
humidity = output_data['Humidity'].astype(float)
humidity

0      72.0
1      74.0
2      73.0
3      39.0
4      76.0
       ... 
543    32.0
544     4.0
545    62.0
546    93.0
547    64.0
Name: Humidity, Length: 548, dtype: float64

In [6]:
# Max intensity for heat map
max_humidity = humidity.max()

In [7]:
# Adding Heatmap layer to map
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations_df, weights = humidity, max_intensity = max_humidity)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Dropping null values in dataframe
output_data_clean = output_data.dropna()
output_data_clean.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [9]:
# Narrowing down cities and fitting ideal weather conditions
conditions_df = output_data_clean.loc[(output_data_clean['Max Temp'] < 80) & (output_data_clean['Max Temp'] > 70) & (output_data_clean['Wind Speed'] < 10) & (output_data_clean['Cloudiness'] == 0)]
conditions_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = pd.DataFrame(conditions_df,columns=['City_ID','City','Cloudiness','Country','Date','Humidity','Lat','Lng','Max Temp','Wind Speed'])
hotel_df['Hotel Name'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [11]:
# Initialising parameters
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

params = {
    "keyword":target_search,
    "radius":target_radius,
    "type":target_type,
    "key":g_key    
}

# Looping through dataframe

for index, row in hotel_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lng = row['Lng']
    
    # Updating location key value
    params['location'] = f'{lat},{lng}'
    
    # Making a request and printing url
    response = requests.get(url, params=params).json()
    pprint(response)
    
    # Exception handling
    try:
        hotel_df.loc[index,"Hotel Name"] = response['results'][0]['name']
    except:
        print('Missing field/result... skipping.')

{'html_attributions': [],
 'next_page_token': 'CrQCIQEAAFt-9oDD0DmEOJEqAdDZApeMdJWzQ2N-qu8eIUyPp0wvB6KXQstVZep-15jpdl0XMf5Nrc7_IKj78TTjUgM-bTmBqYCzfxFr_uSUvRJSvZoYfk_Nd18aWkOV-XAqoUt3GuwvR_mOp4DqMj2seeiqbeCMjIIIqYs93oZzHJNJiyNG1EFwHyZBUs386Am9uH34ceMfprNivZbNFxo6kJBqDlWdBeEjvNuPtk6fsBzQDlhuUnKU54ZUiBuPIseNpZR3O6qoWVkD5rdnROWFfbuvDWaS57rq6e4MQI_0JeVmP971CMDJLu39OnKlYDHwyxwR2pQv_S6MAnFPQY5qdYkCmgBSnEq-K1BrGXh5O5FzMQ46oYNhvLuNUvhXq6nj97XQgf_-q6HudDV2RgiyStyPUFoSENfFD6hFQ7rgTnRbIo69G4YaFIV9y7t4oArmPnsqzBpGpmNOvqgB',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.6183873, 'lng': -46.0491482},
                           'viewport': {'northeast': {'lat': -20.61700662010728,
                                                      'lng': -46.04779677010728},
                                        'southwest': {'lat': -20.61970627989272,
                                                      'lng': -46.05049642989272}}},
              'icon': 'ht

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -30.1778807, 'lng': -50.2035006},
                           'viewport': {'northeast': {'lat': -30.17650502010729,
                                                      'lng': -50.20220167010728},
                                        'southwest': {'lat': -30.17920467989273,
                                                      'lng': -50.20490132989272}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Hotel Castelo',
              'photos': [{'height': 2340,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/108306397827095080979">Daiane '
                                                'Santos</a>'],
                          'photo_reference': 'CmRaAAAAohfJoWlwk399oq3X41UJAW8XdizXR7nLtwkrjXn0M

{'html_attributions': [],
 'next_page_token': 'CrQCIQEAAMzBDQCuSpnk80Z284X2MqJd6RidQtHanpNToubrLhNjW7ITtG9CG9_KB08937vUxGBVH8gamVLH5J8Z5iJTGmkB59bUoLBUmmxZZQV_ZxMPu0YI0r737EGvZ6T-5QxfbkqkL9AcLrHjfAyy6_nzEZxTLXpxzEayVtTL812Djhdh-OHQABpe24vlZ2-sT0OGG0P8USEcWxDrfeevDWZI3X3FTo4XcZQB8BRv69vytG_9EH4e_t0OYREKM3131T_Iv7zBvPIfxO5LZ-WCBrC1-AeI8_4eo7e0fvpJAqk77xZ5FdoDUh8X9depFtu2a9YJRI2gCenQgJWfo-pYhBNxHWHIjRaLzmpv4wJlolMVnkl6nzbqfLwCow3JxmC8IzMGlHmiplGNT_F51GGWxFVf3UYSEBMBO8zWJH5dkmTZCUJvQvcaFLItw08ndKbJ2WpdnkFhaNDnY5zV',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.3000259, 'lng': 44.2679367},
                           'viewport': {'northeast': {'lat': -20.29871172010727,
                                                      'lng': 44.26928077989272},
                                        'southwest': {'lat': -20.30141137989272,
                                                      'lng': 44.26658112010728}}},
              'icon': 'https

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 15.3232474, 'lng': 74.5068496},
                           'viewport': {'northeast': {'lat': 15.32459932989272,
                                                      'lng': 74.50812037989272},
                                        'southwest': {'lat': 15.32189967010728,
                                                      'lng': 74.50542072010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Dandeli Lake County',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 853,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/100842033022261574539">woodpecker '
                                                'jungle camp</a>'],
                          'photo_ref

{'html_attributions': [],
 'results': [{'business_status': 'CLOSED_TEMPORARILY',
              'geometry': {'location': {'lat': 22.9096939, 'lng': 96.5038617},
                           'viewport': {'northeast': {'lat': 22.91113002989272,
                                                      'lng': 96.50508842989272},
                                        'southwest': {'lat': 22.90843037010728,
                                                      'lng': 96.50238877010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Mount Mogok Hotel',
              'permanently_closed': True,
              'photos': [{'height': 607,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/111175695056602902702">A '
                                                'Google User</a>'],
                          'photo_reference': 'CmR

{'html_attributions': [],
 'next_page_token': 'CrQCIQEAAHw0dtWlq8ruFQm-bGdQTJ6JqShT-30vSUMeUKpw3cRWnFSnR4QADhgtO0iiVrbww24atqwd1XKMdnqrnfo6S16xRlOYpVCMv391O1n6D-IYc9qDAGNGJZrUCDq1GGdxy_8po2oLKLr5c8HxvFpbjrUAy-njPwaqyux6qY4k3s0gc_FZ-45D4SeIq67xt-yrYtCQL-iFNmcYHsXrqzoktHWlnCf5NVLjzRj4qUDiH1upoime9q0MqvjDKO3kJ3WjRhShiyewO47iogA0SPFKp-YY6Ibf23YSfDcS4HaOh9TrCC81ANch6WQO2VQWGuj5I94dzsFe3OxqK1bIBlnTQbSHrKY3F19Olx5-niw72VmLgQbKB5S33987MaTjg02AA5zfV5bKlzHRsKMyMnlVBLYSEEkH_s9gW_y9Qqu8V2Hh6e0aFA7U8wv2RdSy9jUplZUXtaI8femJ',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 32.8851325, 'lng': 59.2167535},
                           'viewport': {'northeast': {'lat': 32.88649142989272,
                                                      'lng': 59.21818162989271},
                                        'southwest': {'lat': 32.88379177010728,
                                                      'lng': 59.21548197010727}}},
              'icon': 'https://

In [12]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Pousada Buena Vista
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,"Hotel Marchica Lagoon Resort, Nador Morocco"
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mount Mogok Hotel
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Ghaem Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))